In [1]:
%load_ext Cython
%time
%cython

ERROR:root:Line magic function `%cython` not found (But cell magic `%%cython` exists, did you mean that instead?).


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [2]:
#import needed packages
from gensim.models import doc2vec

import re
import pandas as pd
import numpy as np
import copy

# nltk processing
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews

/Users/yvonnelau/anaconda/lib/python2.7/site-packages/cffi/model.py:526: UserWarning: 'point_conversion_form_t' has no values explicitly defined; next version will refuse to guess which integer type it is meant to be (unsigned/signed, int/long)
  % self._get_c_name())


In [3]:
import mysql.connector
import yaml

cfg = yaml.safe_load(open('_inc.yaml'))
cnx = mysql.connector.connect(user=cfg['mysql']['user'], password=cfg['mysql']['pwd'],
                            host=cfg['mysql']['server'], database=cfg['mysql']['db'])

In [6]:
# load games dataset
games = pd.read_sql_query("select * from tblGame;", cnx)
# load tv shows dataset
tv = pd.read_sql_query("select * from tblTVShow;", cnx)
# load movies dataset
movies = pd.read_sql_query("select * from tblMovie;", cnx) 
# load Reviews
reviews = pd.read_sql_query("select * from tblReview;", cnx)


In [10]:
reviews.head(5)

,gameID,movieID,tvShowID,author,publication,text,score,date,thumbsUp,thumbsDown,reviewType,uniqueID
0,1,-1,-1,"""""",Areajugones,"""Ginger: Beyond the Crystal is a very good pla...",69,"""2016-11-05 00:00:00",0,0,c,1
1,1,-1,-1,"""""",Areajugones,"""Ginger: Beyond the Crystal is a very good pla...",69,"""2016-11-05 00:00:00",0,0,c,1
2,1,-1,-1,"""""",Areajugones,"""Ginger: Beyond the Crystal is a very good pla...",69,"""2016-11-05 00:00:00",0,0,c,1
3,1,-1,-1,,Areajugones,Ginger: Beyond the Crystal is a very good plat...,69,2016-11-05 00:00:00,0,0,c,1
4,1,-1,-1,,Hobby Consolas,"It\s not a bad game at all, but it has technic...",67,2016-11-02 00:00:00,0,0,c,1


In [27]:
ids = games[['uniqueID']].append(movies[['uniqueID']]).append(tv[['uniqueID']])

In [11]:
# subset for just critic reviews to build a model
subset = reviews[reviews['reviewType']=='c']
subset.head(5)

,gameID,movieID,tvShowID,author,publication,text,score,date,thumbsUp,thumbsDown,reviewType,uniqueID
0,1,-1,-1,"""""",Areajugones,"""Ginger: Beyond the Crystal is a very good pla...",69,"""2016-11-05 00:00:00",0,0,c,1
1,1,-1,-1,"""""",Areajugones,"""Ginger: Beyond the Crystal is a very good pla...",69,"""2016-11-05 00:00:00",0,0,c,1
2,1,-1,-1,"""""",Areajugones,"""Ginger: Beyond the Crystal is a very good pla...",69,"""2016-11-05 00:00:00",0,0,c,1
3,1,-1,-1,,Areajugones,Ginger: Beyond the Crystal is a very good plat...,69,2016-11-05 00:00:00,0,0,c,1
4,1,-1,-1,,Hobby Consolas,"It\s not a bad game at all, but it has technic...",67,2016-11-02 00:00:00,0,0,c,1


In [12]:
# get a unique review corpus
review_docs = subset.groupby(['uniqueID'])['text'].apply(lambda x: ''.join(x)).reset_index()
review_docs.head(5)

,uniqueID,text
0,1,"""Ginger: Beyond the Crystal is a very good pla..."
1,2,JoJo’s Bizarre Adventure: Eyes in Heaven featu...
2,3,The Crew does a lot of things right. Not only ...
3,4,Playing as Batgirl is not quite as interesting...
4,5,The frantic nature of the gameplay coupled wit...


In [23]:
review_docs = pd.merge(ids[['uniqueID']], review_docs, on = "uniqueID", how = "left")
# substitute null for ""
review_docs.loc[np.any(review_docs.isnull(), axis = 1),'text'] = ""

In [28]:
stopwrds = stopwords.words('english')
# aux function to clean up text
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = re.sub('[^\w\s]',' ', sentence)
    sentence = re.sub('_',' ', sentence)
    sentence = re.sub('\d+',' ', sentence)
    cleaned = ' '.join([w for w in sentence.split() if not w in stopwrds])
    cleaned = ' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    cleaned = ' '.join([w for w in cleaned.split() if not len(w)<=2 ])
    cleaned = cleaned.strip()
    return cleaned

In [29]:
# clean up review docs, add utf8 encoding
review_docs['textClean'] = review_docs.apply(lambda row: cleaning_text(row['text'].encode("utf8")), axis=1)

In [30]:
# define a few functions for doc2vec processing
def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

# MyDocs reading from a data frame
class MyDocs(object):
    def __iter__(self):
        for i in range(review_docs.shape[0]):
            yield doc2vec.LabeledSentence(words=split_sentence(review_docs.iloc[i,2]), tags=['%s' % review_docs.iloc[i,0]])

In [31]:
# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4)
model.save("review.model")

In [32]:
# testing similar words
print model.most_similar(positive=["cheap", "zelda"], negative=["slow"], topn=5)

[('princess', 0.5448439121246338), ('inexpensive', 0.5323823094367981), ('eshop', 0.5036611557006836), ('qforce', 0.49544674158096313), ('deluxe', 0.4932363033294678)]


In [33]:
#save a pickle file
import pickle

with open('models/model_critic_only.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [35]:
# get a unique summary corpus
summary_docs = games[['uniqueID','summary']].append(movies[['uniqueID', 'summary']]).append(tv[['uniqueID','summary']])
summary_docs.head(5)

,uniqueID,summary
0,1,"A world destroyed by a crystal explosion, a my..."
1,2,"Featuring a new two-on-two battle system, larg..."
2,3,Strap in for a ride that will find you infiltr...
3,4,When The Joker kidnaps Commissioner Gordon and...
4,5,The world has been dominated by dark forces si...


In [36]:
# clean up summary docs, add utf8 encoding when there is no None Type
summary_docs['textClean'] = summary_docs.apply(lambda row: row['summary'] if row['summary'] is None else cleaning_text(row['summary'].encode("utf8")), axis=1)
summary_docs.loc[np.any(summary_docs.isnull(), axis = 1), 'textClean'] = ""

In [37]:
# define class for summary Doc2vec

# MyDocs reading from a data frame
class MyDocs_summary(object):
    def __iter__(self):
        for i in range(summary_docs.shape[0]):
            yield doc2vec.LabeledSentence(words=split_sentence(summary_docs.iloc[i,2]), tags=['%s' % summary_docs.iloc[i,0]])

In [42]:
# Train the doc2vec model on summary
mydocs_summary = MyDocs_summary()
model_summary = doc2vec.Doc2Vec(mydocs_summary, size = 200, window = 8, min_count = 5, workers = 4)
model_summary.save("summary.model")

In [43]:
# testing similar words
print model_summary.most_similar(positive=["cheap", "zelda"], negative=["slow"], topn=5)

[('peter', 0.9542904496192932), ('hobbit', 0.9482103586196899), ('author', 0.9464740753173828), ('robert', 0.9445735216140747), ('penultimate', 0.9298714399337769)]


In [44]:
# Combining both summary and critic reviews doc2vec :model_summary and model 
summary_critics_docvecs = np.zeros(shape=(len(model_summary.docvecs),400))
for i in range(len(model_summary.docvecs)):
    summary_critics_docvecs[i] = np.concatenate((model_summary.docvecs[i],model.docvecs[i]), axis = 0)

In [45]:
#save pickle file
import pickle
with open('summary_critics_docvecs.pickle', 'wb') as handle:
    pickle.dump(summary_critics_docvecs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [334]:
import numpy as np
import pandas as pd
import pickle
import copy
####### TEST REC FROM HERE
# For the app, the code for the recommendation would load a pickle from this cell

#to load a pickle file
with open('summary_critics_docvecs.pickle', 'rb') as f:
    summary_critics_docvecs = pickle.load(f)
    
# Auxiliary functions for simple recommendation system 

# Calculate cosine similarity between two vecotrs 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 

# return top_n values from a list
def top_n_index(l,n):
    return sorted(range(len(l)), key=lambda i: l[i])[-(n+1):-1] #-1 to take off the own product from the returned index list

def content_recommend(item_id, top_n, inputs):
    input_vec = inputs[item_id - 1]
    
    #compute similarity array
    sim_array = map(lambda v: cossim(input_vec, v), inputs)
    
    # recommendation's index 
    recommendation_index = top_n_index(sim_array, top_n)
    
    # recommendation's unique id
    recommendation_unique_id = [i+1 for i in recommendation_index]
    
    # recommendation's cossim values
    recommendation_cossim = [sim_array[i] for i in recommendation_index]
    
    return zip(recommendation_unique_id, recommendation_cossim)

In [46]:
############# Testing content-based recommendation

In [53]:
products_dataset = games[['uniqueID', 'name']].append(movies[['uniqueID','name']]).append(tv[['uniqueID','name']])

In [113]:
# returns a dataframe with top_n products with cosine similarity score
def recommend(item_id, top_n, products_dataset, inputs):
    input_vec = inputs[item_id - 1]
    
    #compute similarity array
    sim_array = map(lambda v: cossim(input_vec, v), inputs)
    
    # recommendation's index and unique_id
    recommendation_index = top_n_index(sim_array, top_n)
    recommendation_unique_id = [i+1 for i in top_n_index(sim_array, top_n)]
    
    # return list of products 
    top_products = copy.deepcopy(products_dataset[products_dataset['uniqueID'].isin(recommendation_unique_id)])
    
     # initialize an empty column for cosine similarity
    top_products['cossim']=0
    
    for i in range(len(recommendation_index)):
        # YL note: avoid top_products['cossim'].loc[index[i],] = sim_array[index[i]], results in error
        top_products.loc[top_products['uniqueID']==recommendation_unique_id[i],'cossim'] = sim_array[recommendation_index[i]]
    return top_products

In [140]:
products_dataset.loc[products_dataset['name'].str.contains('Quantico')]

,uniqueID,name
1071,26958,Quantico: Season 1


In [141]:
id = 26958
print products_dataset[products_dataset['uniqueID']==id]['name']
recommend(id, 10, products_dataset, summary_critics_docvecs)

1071    Quantico: Season 1
Name: name, dtype: object


,uniqueID,name,cossim
1655,1656,Syberia II,0.755504
5711,5712,Dinotopia: The Sunstone Odyssey,0.762167
15569,15570,The Flock,0.752312
15667,15668,Whispering Willows,0.769067
910,21327,Jerichow,0.752076
2507,22924,Shadow Dancer,0.764751
4240,24657,Big Sky,0.767464
4483,24900,Shut In,0.746143
4987,25404,The Son of No One,0.751610
1882,27769,Manhattan: Season 2,0.752934
